In [61]:
import tarfile
import os
import json
import pandas as pd
import re
!pip install tensorflow

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [69]:
goals_list = [ #Goals provided by ChatGPT
    "Increase website traffic by 20% within the next 6 months through SEO optimization",
    "Achieve a customer satisfaction rating of 90% or above within the next quarter by improving customer service processes",
    "Reduce customer churn rate by 15% by implementing a loyalty program within the next year",
    "Increase sales revenue by 25% within the next fiscal year by expanding into new markets",
    "Improve employee productivity by 15% by providing additional training and resources within the next 6 months",
    "Launch a new product line within the next 9 months, targeting a 10% market share within the first year",
    "Decrease product defects by 20% by implementing a quality control system within the next 6 months",
    "Increase social media engagement by 30% within the next quarter by posting more interactive content",
    "Reduce operational costs by 10% within the next year through process optimization and automation",
    "Achieve a net promoter score (NPS) of 50 or above within the next 6 months by improving product satisfaction",
    "Expand the email subscriber list by 50% within the next 3 months through targeted marketing campaigns",
    "Improve employee retention rate by 20% within the next year by enhancing workplace culture and benefits",
    "Launch a company-wide sustainability initiative within the next 6 months to reduce carbon footprint by 15%",
    "Increase market share by 10% within the next fiscal year through competitive pricing strategies",
    "Reduce average customer response time by 25% within the next quarter by implementing a new ticketing system",
    "Improve brand awareness by 40% within the next year through strategic partnerships and sponsorships",
    "Achieve ISO 9001 certification within the next 12 months by meeting quality management standards",
    "Increase customer lifetime value by 15% within the next 6 months through personalized marketing campaigns",
    "Launch a mobile app within the next 9 months to provide customers with a more convenient shopping experience",
    "Reduce shipping times by 20% within the next quarter by optimizing logistics and fulfillment processes",
    "Improve employee satisfaction scores by 15% within the next year through regular feedback sessions and perks",
    "Achieve a 95% on-time delivery rate within the next 6 months by streamlining supply chain operations",
    "Expand into international markets within the next fiscal year, targeting at least 3 new countries",
    "Increase repeat purchase rate by 20% within the next quarter through customer loyalty programs",
    "Implement a flexible work-from-home policy within the next month to improve employee work-life balance",
    "Achieve a 50% reduction in customer complaints within the next 6 months through better product training for staff",
    "Launch a company blog within the next 3 months to provide valuable content and improve SEO",
    "Increase customer referrals by 25% within the next quarter through incentivized referral programs",
    "Upgrade IT infrastructure within the next year to improve system reliability and security",
    "Implement regular performance evaluations within the next month to track employee progress and development",
    "Reduce product return rate by 15% within the next 6 months through improved product descriptions and customer support",
    "Launch a social responsibility program within the next year to give back to the community and improve brand perception",
    "Increase average order value by 10% within the next quarter through upselling and cross-selling strategies",
    "Improve website conversion rate by 20% within the next 6 months through A/B testing and optimization",
    "Expand customer support hours within the next month to provide 24/7 assistance",
    "Achieve a 20% increase in employee engagement scores within the next year through team-building activities and recognition programs",
    "Launch a customer feedback initiative within the next quarter to gather insights for product improvement",
    "Reduce energy consumption by 15% within the next fiscal year through energy-efficient upgrades",
    "Implement a mentorship program within the next 6 months to foster employee growth and development",
    "Increase online reviews by 30% within the next quarter through proactive review solicitation",
    "Enhance the user experience of the company website within the next 6 months to reduce bounce rate",
    "Achieve a 20% increase in customer retention within the next year through personalized retention strategies",
    "Launch a referral program within the next quarter to incentivize existing customers to refer new business",
    "Improve employee training programs within the next 3 months to ensure staff are equipped with necessary skills",
    "Achieve a 50% reduction in inventory holding costs within the next fiscal year through better inventory management practices"
]

print(len(goals_list))

smart_goals = [
    "Lose 10 pounds in 3 months by exercising 4 times a week and following a healthy diet",
    "Save $5000 in one year by setting aside $417 each month",
    "Read 12 books in the next 12 months, one book per month",
    "Run a 5K race in 6 months by training three times a week",
    "Learn to play 5 songs on the guitar within the next 4 months by practicing daily",
    "Increase monthly sales by 15% in the next quarter through targeted marketing campaigns",
    "Complete an online course on data science within 6 months by dedicating 5 hours a week",
    "Write a 50,000-word novel in 6 months by writing 2000 words every week",
    "Improve public speaking skills by attending 10 Toastmasters meetings in the next 6 months",
    "Launch a personal blog and publish 20 articles in the next 4 months",
    "Reduce monthly electricity bills by 20% in the next 6 months by implementing energy-saving measures",
    "Volunteer 50 hours at a local charity in the next 6 months",
    "Learn conversational Spanish in 6 months by practicing for 30 minutes a day",
    "Increase social media followers by 25% in the next 3 months through consistent content posting",
    "Create a professional portfolio website within 3 months",
    "Complete a half marathon in 9 months by following a training plan",
    "Achieve a 10% increase in website traffic over the next 6 months through SEO improvements",
    "Plant and maintain a vegetable garden and harvest produce within the next 4 months",
    "Meditate for 10 minutes daily for the next 3 months to reduce stress",
    "Organize and declutter the entire house within 2 months, focusing on one room per week",
    "Attend a networking event or conference related to my industry every month for the next year",
    "Create and stick to a weekly meal plan for the next 3 months",
    "Start a side business and make the first $1000 profit within 6 months",
    "Build a personal finance budget and stick to it for 6 months",
    "Achieve a certification in my field within the next 9 months",
    "Improve running speed to complete a mile in under 8 minutes within 3 months",
    "Cook a new recipe every week for the next 3 months",
    "Learn to code in Python and complete 5 projects in the next 6 months",
    "Improve sleep habits by going to bed before 10 PM every night for the next 3 months",
    "Increase water intake to 8 glasses daily for the next 3 months",
    "Complete a 30-day yoga challenge",
    "Host a dinner party for friends within the next 2 months",
    "Increase daily step count to 10,000 steps for the next 3 months",
    "Read and summarize 1 research paper related to my field each week for the next 6 months",
    "Practice piano for 30 minutes a day and learn 5 new pieces in 3 months",
    "Reduce credit card debt by 50% within the next 12 months",
    "Take a photography class and create a portfolio of 50 edited photos within 6 months",
    "Run a 10K race in 6 months by training 4 times a week",
    "Take a family vacation within the next 12 months and save $2000 for the trip",
    "Create a weekly cleaning schedule and stick to it for the next 3 months",
    "Improve handwriting by practicing 20 minutes daily for 2 months",
    "Build a professional network by connecting with 2 new people each week for the next 6 months",
    "Write and send 5 thank-you notes each month for the next 6 months",
    "Achieve 90% accuracy in touch typing within the next 3 months",
    "Learn to swim 1000 meters continuously within 6 months by practicing twice a week",
    "Develop and implement a morning routine for the next 3 months",
    "Successfully follow a daily journal habit for the next 3 months",
    "Paint 10 canvases within the next 6 months",
    "Create a capsule wardrobe by reducing clothing items by 50% within 3 months",
    "Increase daily fruit and vegetable intake to 5 servings for the next 3 months",
    "Write a business plan for a new venture within the next 2 months",
    "Participate in a monthly book club for the next 12 months",
    "Complete a 12-week online fitness program",
    "Achieve a 10% improvement in customer satisfaction scores over the next 6 months",
    "Save $3000 for an emergency fund within the next 12 months",
    "Finish a 30-day drawing challenge",
    "Reduce caffeine intake to one cup of coffee per day for the next 3 months",
    "Take a cooking class and master 10 new recipes within 6 months",
    "Launch an email newsletter and gain 500 subscribers in the next 6 months",
    "Organize and digitize all family photos within 3 months",
    "Create and follow a skincare routine for the next 3 months",
    "Learn and use 3 new vocabulary words daily for the next 2 months",
    "Prepare and file taxes 2 months before the deadline",
    "Take a daily walk for 30 minutes for the next 3 months",
    "Start a gratitude journal and write in it daily for the next 3 months",
    "Design and build a piece of furniture within the next 4 months",
    "Complete a 7-day technology detox",
    "Reach 100% fluency in a language-learning app within 6 months",
    "Save $100 a month by cutting unnecessary expenses for the next 6 months",
    "Attend a workshop or seminar related to my career every quarter for the next year",
    "Create a workout plan and stick to it for the next 3 months",
    "Implement a daily stretching routine for the next 3 months",
    "Improve handwriting by practicing for 15 minutes every day for the next 3 months",
    "Host a virtual game night with friends once a month for the next 6 months",
    "Reduce sugar intake by 50% for the next 3 months",
    "Complete a daily 10-minute mindfulness practice for the next 3 months",
    "Learn and perform a new dance routine within the next 3 months",
    "Write and publish 3 guest blog posts in the next 6 months",
    "Attend a fitness class twice a week for the next 6 months",
    "Read one scientific journal article per week for the next 6 months",
    "Complete a 5-day juice cleanse",
    "Create a vision board and review it weekly for the next 6 months",
    "Enroll in a financial literacy course and complete it within 3 months",
    "Host a charity event and raise $1000 within the next 6 months",
    "Run 3 miles without stopping within 3 months",
    "Create a family photo album and complete it within 4 months",
    "Visit one new place in my city every month for the next year",
    "Bake a new dessert every week for the next 3 months",
    "Achieve 8 hours of sleep every night for the next 3 months",
    "Complete a 30-day fitness challenge",
    "Develop a new hobby and practice it weekly for the next 6 months",
    "Reduce screen time to 2 hours a day for the next 3 months",
    "Conduct a monthly budget review for the next 12 months",
    "Learn to sew and complete 5 projects within 6 months",
    "Achieve a specific work performance goal in the next quarter",
    "Complete a weekend hiking trip within the next 3 months",
    "Grow 3 new plants in the garden within the next 4 months",
    "Learn a new sport and practice it weekly for the next 6 months",
    "Create a personal brand and develop a strategy within the next 6 months",
    "Write a short story and submit it to a competition within 4 months"
]


print(len(smart_goals))

smart_goals_two = [
    "Lose 10 pounds in 3 months by exercising 4 times a week and following a healthy diet",
    "Save $5000 in one year by setting aside $417 each month",
    "Read 12 books in the next 12 months, one book per month",
    "Run a 5K race in 6 months by training three times a week",
    "Learn to play 5 songs on the guitar within the next 4 months by practicing daily",
    "Increase monthly sales by 15% in the next quarter through targeted marketing campaigns",
    "Complete an online course on data science within 6 months by dedicating 5 hours a week",
    "Write a 50,000-word novel in 6 months by writing 2000 words every week",
    "Improve public speaking skills by attending 10 Toastmasters meetings in the next 6 months",
    "Launch a personal blog and publish 20 articles in the next 4 months",
    "Reduce monthly electricity bills by 20% in the next 6 months by implementing energy-saving measures",
    "Volunteer 50 hours at a local charity in the next 6 months",
    "Learn conversational Spanish in 6 months by practicing for 30 minutes a day",
    "Increase social media followers by 25% in the next 3 months through consistent content posting",
    "Create a professional portfolio website within 3 months",
    "Complete a half marathon in 9 months by following a training plan",
    "Achieve a 10% increase in website traffic over the next 6 months through SEO improvements",
    "Plant and maintain a vegetable garden and harvest produce within the next 4 months",
    "Meditate for 10 minutes daily for the next 3 months to reduce stress",
    "Organize and declutter the entire house within 2 months, focusing on one room per week",
    "Attend a networking event or conference related to my industry every month for the next year",
    "Create and stick to a weekly meal plan for the next 3 months",
    "Start a side business and make the first $1000 profit within 6 months",
    "Build a personal finance budget and stick to it for 6 months",
    "Achieve a certification in my field within the next 9 months",
    "Improve running speed to complete a mile in under 8 minutes within 3 months",
    "Cook a new recipe every week for the next 3 months",
    "Learn to code in Python and complete 5 projects in the next 6 months",
    "Improve sleep habits by going to bed before 10 PM every night for the next 3 months",
    "Increase water intake to 8 glasses daily for the next 3 months",
    "Complete a 30-day yoga challenge",
    "Host a dinner party for friends within the next 2 months",
    "Increase daily step count to 10,000 steps for the next 3 months",
    "Read and summarize 1 research paper related to my field each week for the next 6 months",
    "Practice piano for 30 minutes a day and learn 5 new pieces in 3 months",
    "Reduce credit card debt by 50% within the next 12 months",
    "Take a photography class and create a portfolio of 50 edited photos within 6 months",
    "Run a 10K race in 6 months by training 4 times a week",
    "Take a family vacation within the next 12 months and save $2000 for the trip",
    "Create a weekly cleaning schedule and stick to it for the next 3 months",
    "Improve handwriting by practicing 20 minutes daily for 2 months",
    "Build a professional network by connecting with 2 new people each week for the next 6 months",
    "Write and send 5 thank-you notes each month for the next 6 months",
    "Achieve 90% accuracy in touch typing within the next 3 months",
    "Learn to swim 1000 meters continuously within 6 months by practicing twice a week",
    "Develop and implement a morning routine for the next 3 months",
    "Successfully follow a daily journal habit for the next 3 months",
    "Paint 10 canvases within the next 6 months",
    "Create a capsule wardrobe by reducing clothing items by 50% within 3 months",
    "Increase daily fruit and vegetable intake to 5 servings for the next 3 months",
    "Write a business plan for a new venture within the next 2 months",
    "Participate in a monthly book club for the next 12 months",
    "Complete a 12-week online fitness program",
    "Achieve a 10% improvement in customer satisfaction scores over the next 6 months",
    "Save $3000 for an emergency fund within the next 12 months",
    "Finish a 30-day drawing challenge",
    "Reduce caffeine intake to one cup of coffee per day for the next 3 months",
    "Take a cooking class and master 10 new recipes within 6 months",
    "Launch an email newsletter and gain 500 subscribers in the next 6 months",
    "Organize and digitize all family photos within 3 months",
    "Create and follow a skincare routine for the next 3 months",
    "Learn and use 3 new vocabulary words daily for the next 2 months",
    "Prepare and file taxes 2 months before the deadline",
    "Take a daily walk for 30 minutes for the next 3 months",
    "Start a gratitude journal and write in it daily for the next 3 months",
    "Design and build a piece of furniture within the next 4 months",
    "Complete a 7-day technology detox",
    "Reach 100% fluency in a language-learning app within 6 months",
    "Save $100 a month by cutting unnecessary expenses for the next 6 months",
    "Attend a workshop or seminar related to my career every quarter for the next year",
    "Create a workout plan and stick to it for the next 3 months",
    "Implement a daily stretching routine for the next 3 months",
    "Improve handwriting by practicing for 15 minutes every day for the next 3 months",
    "Host a virtual game night with friends once a month for the next 6 months",
    "Reduce sugar intake by 50% for the next 3 months",
    "Complete a daily 10-minute mindfulness practice for the next 3 months",
    "Learn and perform a new dance routine within the next 3 months",
    "Write and publish 3 guest blog posts in the next 6 months",
    "Attend a fitness class twice a week for the next 6 months",
    "Read one scientific journal article per week for the next 6 months",
    "Complete a 5-day juice cleanse",
    "Create a vision board and review it weekly for the next 6 months",
    "Enroll in a financial literacy course and complete it within 3 months",
    "Host a charity event and raise $1000 within the next 6 months",
    "Run 3 miles without stopping within 3 months",
    "Create a family photo album and complete it within 4 months",
    "Visit one new place in my city every month for the next year",
    "Bake a new dessert every week for the next 3 months",
    "Achieve 8 hours of sleep every night for the next 3 months",
    "Complete a 30-day fitness challenge",
    "Develop a new hobby and practice it weekly for the next 6 months",
    "Reduce screen time to 2 hours a day for the next 3 months",
    "Conduct a monthly budget review for the next 12 months",
    "Learn to sew and complete 5 projects within 6 months",
    "Achieve a specific work performance goal in the next quarter",
    "Complete a weekend hiking trip within the next 3 months",
    "Grow 3 new plants in the garden within the next 4 months",
    "Learn a new sport and practice it weekly for the next 6 months",
    "Create a personal brand and develop a strategy within the next 6 months",
    "Write a short story and submit it to a competition within 4 months",
    "Learn basic coding in HTML and CSS within 3 months",
    "Create a comprehensive will and estate plan within the next 6 months",
    "Create a daily reading habit and read for 30 minutes each day for the next 6 months",
    "Complete a 10-day meditation retreat within the next 12 months",
    "Improve listening skills by attending 4 active listening workshops in the next 6 months",
    "Reduce plastic use by 50% within the next 6 months",
    "Complete a first aid and CPR certification course within the next 3 months",
    "Write a 1000-word article and get it published in a magazine within 6 months",
    "Complete a photography project with a theme and exhibit the photos within 6 months",
    "Create a personal development plan for the next 12 months and review it quarterly",
    "Attend 8 yoga classes a month for the next 3 months",
    "Learn to bake 5 types of bread within the next 3 months",
    "Create and stick to a morning routine for the next 3 months",
    "Learn to play a new musical instrument within the next 6 months",
    "Organize and host a family reunion within the next 12 months",
    "Achieve 500 hours of volunteer work within the next 12 months",
    "Build a custom piece of furniture within the next 4 months",
    "Improve cardiovascular health by running 4 times a week for the next 6 months",
    "Complete a course on financial management within the next 3 months",
    "Learn to create and edit videos within the next 6 months",
    "Achieve a daily habit of flossing for the next 3 months",
    "Complete an advanced level in a language learning app within the next 6 months",
    "Write and self-publish a book within the next 12 months",
    "Complete a 30-day no-spending challenge",
    "Learn to paint with watercolors and complete 5 paintings within the next 3 months",
    "Take a class on public speaking and give a speech within 6 months",
    "Save $5000 for a down payment on a house within the next 12 months",
    "Plan and complete a home renovation project within 6 months",
    "Create a capsule wardrobe and donate unused clothes within the next 3 months",
    "Achieve a 10% increase in productivity at work by implementing time management strategies within 6 months",
    "Start a blog and write 50 posts within the next 12 months",
    "Learn to play chess and participate in a local tournament within the next 6 months",
    "Attend a monthly industry networking event for the next year",
    "Complete a 10,000-piece jigsaw puzzle within 6 months",
    "Read 5 classic novels within the next 6 months",
    "Take a graphic design course and create a portfolio within 6 months",
    "Achieve 100% attendance at work or school for the next 6 months",
    "Learn and practice a new programming language within 6 months",
    "Reach a savings goal of $10,000 within the next 12 months",
    "Complete a triathlon within the next 12 months by following a training plan",
    "Create a successful blog and reach 10,000 monthly visitors within 6 months",
    "Reduce carbon footprint by 20% in the next 6 months through sustainable practices",
    "Attend a monthly creative writing group for the next 12 months",
    "Complete a woodworking project within the next 4 months",
    "Start a podcast and publish 10 episodes within the next 6 months",
    "Achieve a professional certification within the next 9 months",
    "Create a passive income stream of $500 a month within the next 12 months",
    "Reduce screen time to 1 hour a day for the next 3 months",
    "Practice mindfulness meditation for 20 minutes daily for the next 3 months",
    "Attend a personal development seminar every quarter for the next year",
    "Complete a 10-day detox diet within the next 3 months",
    "Create and follow a skin care routine for the next 3 months",
    "Learn and master a new software tool within the next 6 months",
    "Write a letter to a different friend or family member each month for the next year",
    "Complete a 5-day social media detox",
    "Cook a meal from a different cuisine every week for the next 6 months",
    "Achieve a daily goal of 8 hours of sleep for the next 3 months",
    "Learn to create a mobile app and publish it within 6 months",
    "Reduce sugar intake to under 25 grams daily for the next 3 months",
    "Complete a daily fitness challenge for the next 3 months",
    "Start and maintain a weekly family game night for the next 6 months",
    "Learn basic first aid and CPR within the next 3 months",
    "Write a script and produce a short film within the next 6 months",
    "Complete a personal finance course within the next 3 months",
    "Achieve a goal of reading 20 pages a day for the next 6 months",
    "Create and follow a weekly cleaning schedule for the next 3 months",
    "Improve posture by practicing daily exercises for the next 3 months",
    "Start a YouTube channel and upload 12 videos in the next 6 months",
    "Learn to cook 10 new healthy recipes within the next 3 months",
    "Complete a 10-day digital detox",
    "Start a side business and achieve $500 in profit within the next 6 months",
    "Create and follow a detailed meal plan for the next 3 months",
    "Improve flexibility by doing yoga 3 times a week for the next 6 months",
    "Write and send 10 handwritten letters in the next 3 months",
    "Learn to manage stress by practicing meditation daily for the next 3 months",
    "Save $200 a month by reducing unnecessary expenses for the next 6 months",
    "Learn to play a new board game and host a game night within the next 3 months",
    "Achieve 80% proficiency in a new language within the next 6 months",
    "Improve cardiovascular fitness by doing HIIT workouts 3 times a week for the next 3 months",
    "Complete a 10-day green smoothie challenge",
    "Plan and go on a weekend getaway within the next 6 months",
    "Improve time management by using a planner daily for the next 3 months",
    "Create a personal mission statement and review it weekly for the next 3 months",
    "Achieve 100% compliance with flossing daily for the next 3 months",
    "Learn to create and maintain a website within the next 6 months",
    "Achieve a daily habit of drinking 8 glasses of water for the next 3 months",
    "Complete a self-defense course within the next 3 months",
    "Create and stick to a family budget for the next 6 months",
    "Learn to bake and decorate a cake within the next 3 months",
    "Achieve a personal best in a sport or hobby within the next 6 months",
    "Complete a 7-day gratitude challenge",
    "Read a biography of a famous person every month for the next 6 months",
    "Learn to knit or crochet and complete a project within the next 3 months",
    "Complete a 30-day no sugar challenge",
    "Plan and host a themed dinner party within the next 3 months",
    "Improve running endurance by running 3 times a week for the next 3 months",
    "Complete a 5-day water fast",
    "Create and follow a skincare routine for the next 3 months",
    "Learn to play a musical instrument within the next 6 months",
    "Improve sleep quality by going to bed and waking up at the same time every day for the next 3 months"
]

print(len(smart_goals_two))

smart_criteria_list = [ #ChatGPT classified goals as the following
    'SMART', 'SMART', 'MART', 'MART', 'MART',
    'SMART', 'MART', 'MART', 'MART', 'MART',
    'SMART', 'SMART', 'SMART', 'MART', 'MART',
    'MART', 'MART', 'SMART', 'SMART', 'MART',
    'MART', 'MART', 'SMAT', 'MART', 'MART',
    'MART', 'SMART', 'MART', 'SMAT', 'MAR',
    'MART', 'SMAT', 'SMAT', 'MART', 'MART',
    'MART', 'SMART', 'MART', 'SMAT', 'MART',
    'MART', 'SMART', 'MART', 'SMART', 'MART',
]

"""
for x in range(len(smart_criteria_list)):

    if 'S' in smart_criteria_list[x]:
        tst = smart_criteria_list[x]
        tst = tst.replace('S', '')
        smart_criteria_list[x] = tst

    if 'M' in smart_criteria_list[x]:
        tst = smart_criteria_list[x]
        tst = tst.replace('M', '')
        smart_criteria_list[x] = tst
    if 'A' in smart_criteria_list[x]:
        tst = smart_criteria_list[x]
        tst = tst.replace('A', '')
        smart_criteria_list[x] = tst

    if 'R' in smart_criteria_list[x]:
        tst = smart_criteria_list[x]
        tst = tst.replace('R', '')
        smart_criteria_list[x] = tst

    if 'T' in smart_criteria_list[x]:
        tst = smart_criteria_list[x]
        tst = tst.replace('T', '')
        smart_criteria_list[x] = tst
"""

print(smart_criteria_list)

45
100
200
['SMART', 'SMART', 'MART', 'MART', 'MART', 'SMART', 'MART', 'MART', 'MART', 'MART', 'SMART', 'SMART', 'SMART', 'MART', 'MART', 'MART', 'MART', 'SMART', 'SMART', 'MART', 'MART', 'MART', 'SMAT', 'MART', 'MART', 'MART', 'SMART', 'MART', 'SMAT', 'MAR', 'MART', 'SMAT', 'SMAT', 'MART', 'MART', 'MART', 'SMART', 'MART', 'SMAT', 'MART', 'MART', 'SMART', 'MART', 'SMART', 'MART']


# New Section

In [63]:
!pip install tensorflow


In [70]:
smart_criteria_series = pd.Series(smart_criteria_list)

# Convert the list into a pandas Series
#goals_series = pd.Series(goals_list)

#goals_series = pd.Series(smart_goals)
goals_series = pd.Series(smart_goals_two)

#tags_df = pd.DataFrame(index=goals.index, columns=['Specific', 'Measure', 'Achievable', 'Relevant', 'Time-Bound'])
tags_df = pd.DataFrame(index=goals_series.index, columns=['Specific', 'Measure', 'Achievable', 'Relevant', 'Time-Bound'])

words_in_specific_goals = ["finish", "complete", "sleep", "work", "job",
                           " to", "for", "give", "plan"]
                           #Adding numbers decreases the specifics of it by .13, but
                           #a lot of things that are specific have numbers
                           #through decreases it by 0.05%, "improv"
                           #Adding "within" decreases it by .13
words_in_measurable_goals = ["amount", "time", "due", "compar", "measure", "0",
                             "in", "1", "2", "3", "4", "5", "6", "7", "8", "9", "%"]
words_in_achievable_goals = ["easy", "hobby", "hobbies", "did", "have", "gym",
                             "exercise", "small", "skill", "time", "within",
                             "month", "year", "week", "health"]
words_in_relevant_goals = ["need", "have", "learn", "hobby", "hobbies", "music",
                           "job", "assign", "task", "personal", "I", "my",
                           "increase", "org", "financ", "money", "make", "family",
                           "friend", "corp", "community", "product", "company",
                           "customer", "culture"]
words_in_timebound_goals = ["week", "month", "day", "year", "within", "time",
                            "tomorrow", "next", "date", "sec", "min",
                            "hour", "dur", "deadline"]

# Example: Assign tags based on goal characteristics (you can adjust this based on your criteria)

for index, goal in goals_series.items():

    tags_df.loc[index, 'Specific'] = ""
    tags_df.loc[index, 'Measure'] = ""
    tags_df.loc[index, 'Achievable'] = ""
    tags_df.loc[index, 'Relevant'] = ""
    tags_df.loc[index, 'Time-Bound'] = ""


    for word in words_in_specific_goals:
        if word in goal.lower():
            tags_df.loc[index, 'Specific'] = "S"
            break

    for word in words_in_measurable_goals:
        if word in goal.lower():
          tags_df.loc[index, 'Measure'] = "M"
          break

    for word in words_in_achievable_goals:
        if word in goal.lower():
          tags_df.loc[index, 'Achievable'] = "A"
          break

    for word in words_in_relevant_goals:
      if word in goal.lower():
         tags_df.loc[index, 'Relevant'] = "R"
         break

    for word in words_in_timebound_goals:
        if word in goal.lower():
          tags_df.loc[index, 'Time-Bound'] = "T"
          break


tags_list = tags_df.to_dict(orient='records')

predictions = []
for dic in tags_list:
    #print(dic)
    goal_type = dic['Specific'] + dic['Measure'] + dic['Achievable'] + dic['Relevant'] + dic['Time-Bound']
    predictions.append(goal_type)

#print(predictions)
#print(smart_criteria_series)

print(len(predictions))
cnt = 0
for x in range(len(predictions)):
    if predictions[x] == "SMART":
        cnt += 1
    #else:
    #    print(smart_goals[x] + " --> " + predictions[x])
print("Accuracy: " + (str)(round(cnt / len(predictions), 2)))
print()
print()
print()

"""
correct_preds = []
t = 0
for x in range(len(goals_list)):
  if(predictions[x] == smart_criteria_series[x]):
    correct_preds.append("1")
    #print(goals_list[x] + " --> " + predictions[x])
  else:
    correct_preds.append("0")
    #if(t > 5):
    #  print(goals_list[x] + "\nPrediction: " + predictions[x] + "\nResult: " + smart_criteria_series[x])
    #  print()
    #t += 1

#print(correct_preds)
cnt = 0
for i in correct_preds:
    if i == '1':
        cnt += 1

accuracy = cnt / 45
print("Accuracy: " + (str)(round(accuracy, 2)))
"""
#words_in_specific_goals = ["finish", "complete", "eat", "sleep", "work", "job",
#                           " to", "for", "give"] #through decreases it by 0.05% , "improv"


200
Accuracy: 0.23





'\ncorrect_preds = []\nt = 0\nfor x in range(len(goals_list)):\n  if(predictions[x] == smart_criteria_series[x]):\n    correct_preds.append("1")\n    #print(goals_list[x] + " --> " + predictions[x])\n  else:\n    correct_preds.append("0")\n    #if(t > 5):\n    #  print(goals_list[x] + "\nPrediction: " + predictions[x] + "\nResult: " + smart_criteria_series[x])\n    #  print()\n    #t += 1\n\n#print(correct_preds)\ncnt = 0\nfor i in correct_preds:\n    if i == \'1\':\n        cnt += 1\n\naccuracy = cnt / 45\nprint("Accuracy: " + (str)(round(accuracy, 2)))\n'

In [65]:
"""
Increase repeat purchase rate by 20% within the next quarter through customer loyalty programs
Implement a flexible work-from-home policy within the next month to improve employee work-life balance
Achieve a 50% reduction in customer complaints within the next 6 months through better product training for staff
Enhance the user experience of the company website within the next 6 months to reduce bounce rate
Launch a referral program within the next quarter to incentivize existing customers to refer new business
"""

"""
64% if I give it models that are SMAT, SMA, SA, etc.

54% if I give it models that are only SMAT
"""

'\n64% if I give it models that are SMAT, SMA, SA, etc.\n\n54% if I give it models that are only SMAT\n'